In [33]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Part 2: How should we deal with causality?

## Low dimensional settings
When statisticians talk about bias, adjustment is often the go-to solution. When you have a **low dimensional dataset**, *i.e.*, you only have a limited number of features, it is often possible to adjust for confounding variables. This is done by either stratifying (taking the average of the effects X on Y over each of the confounding groups) or by including the confounder variable in the model. 

For example, if you are interested in the effect of a gene expression on a survival rate, as in [Uhlen *et al.*, 2018](https://www.science.org/doi/10.1126/science.aan2507?url_ver=Z39.88-2003&rfr_id=ori:rid:crossref.org&rfr_dat=cr_pub%20%200pubmed), you might want to adjust for the age of the patient, as older patients are more likely to die sooner<sup>*</sup>.

<sup>*</sup> Interestingly, Uhlen *et al.* did not adjust for age in their 2018 Science publication. A highly interesting critical review of their methods can be found [here](https://www.biorxiv.org/content/10.1101/2020.03.16.994038v2).

### Experiment 3: adjusting for confounding variables
The dataset in `data/transcriptomics_pathology.csv` contains metadata and gene expression data on 10 genes for 1000 patients:

In [128]:
data = pd.read_csv('./data/transcriptomics_pathology.csv', index_col=0)
data.head()

,Age,Gender,Gene_1 expression,Gene_2 expression,Survival Years
Patient 1,Young,Woman,0.126586,0.883764,24
Patient 2,Old,Man,1.186966,0.329328,14
Patient 3,Young,Woman,0.180942,0.745151,65
Patient 4,Old,Man,0.695649,0.429752,16
Patient 5,Young,Man,0.189464,0.312411,60


Assuming the two genes' expression levels are independent from each other, but confounded by the age of the patient, build a correctly adjusted linear model to find out which gene(s) likely have a causal effect on the survival rate.

In [132]:
# specify below which variables you want to include in the model
vars_to_include = ['Gene_1 expression', 'Gene_2 expression', 'Age']

# formatting dummy variables
data['Age'].replace(['Young', 'Old'], [0, 1], inplace=True)
data['Gender'].replace(['Man', 'Woman'], [0, 1], inplace=True)

# linear model
lm = LinearRegression()
lm.fit(data.loc[:, vars_to_include], data['Survival Years'])
print(
    """
    Gene 1's expression has an impact on survival with coefficient {},
    Gene 2's expression has an impact on survival with coefficient {}
    """.format(round(lm.coef_[0], 2), round(lm.coef_[1], 2))
    )


    Gene 1's expression has an impact on survival with coefficient -0.72,
    Gene 2's expression has an impact on survival with coefficient -17.18
    


You can quickly check the difference this makes in coefficients when you don't adjust for the age of the patient.

### Experiment 4: All for adjustment, and adjustment for all
While adjustment is a step in the right direction, *i.e.*, the one that acknowledges causality, it can sometimes do more harm than good.

Consider the following example:
**Effects of a New Drug on Blood Pressure**

Imagine a clinical trial testing the efficacy of a new drug designed to lower blood pressure in hypertensive patients. The DAG for this scenario might look like this:

- Treatment: Represents whether a patient received the new drug (e.g., Drug X) or a placebo.
- Blood Pressure: Represents the primary outcome variable, the patient's blood pressure measurement.
- Age: Represents the patient's age, a potential confounding variable associated with both the treatment assignment and blood pressure.
- Exercise: Represents whether the patient regularly partakes in physical activity, which could affect blood pressure independently of the treatment.
- Cholesterol: Represents the cholesterol level of the patient after receiving the treatment. This may impact blood pressure as well.

In [120]:
data = pd.read_csv('./data/blood_pressure.csv', index_col=0)
data.head()

,Treatment,Age,Exercise,Cholesterol,Blood Pressure
Patient 1,Placebo,Young,0,179.690255,109.732353
Patient 2,Treatment,Old,1,168.880741,124.321006
Patient 3,Placebo,Young,0,183.684529,113.626123
Patient 4,Treatment,Young,0,146.573601,97.656936
Patient 5,Placebo,Young,0,196.820208,118.844416


Which variables should you include in your model to estimate the causal effect of the treatment on blood pressure? How does including them influence the coefficient of the treatment variable?

In [126]:
# specify below which variables you want to include in the model
vars_to_include = ['Treatment', 'Age', 'Exercise']

# formatting dummy variables
data['Age'].replace(['Young', 'Old'], [0, 1], inplace=True)
data['Treatment'].replace(['Placebo', 'Treatment'], [0, 1], inplace=True)

# linear model
lm = LinearRegression()
lm.fit(data.loc[:, vars_to_include], data['Blood Pressure'])
print(
    """
    The treatment has an impact on blood pressure with coefficient {},
    age has an impact on blood pressure with coefficient {},
    exercise has an impact on blood pressure with coefficient {}
    """.format(round(lm.coef_[0], 2), round(lm.coef_[1], 2), round(lm.coef_[2], 2))
    )


    The treatment has an impact on blood pressure with coefficient 1.53,
    age has an impact on blood pressure with coefficient 8.08,
    exercise has an impact on blood pressure with coefficient -3.01
    


**Cheatcode:** There is a very handy site called [daggity](http://www.dagitty.net/dags.html) that allows you to draw causal diagrams and check which variables you should adjust for in your analysis. It even allows you to draw in unobserved variables that you know are important, but you can't measure.